In [28]:
import pandas as pd
from sodapy import Socrata

In [30]:
client = Socrata("healthdata.gov","4OmJdnsLoc94vwZg59h9v0Ffo")

In [31]:
results = client.get("uqq2-txqb", limit=2000)


In [32]:
results_df = pd.DataFrame.from_records(results)

In [60]:
results_df = results_df[results_df.city=="DENVER"]

In [62]:
print(results_df)

    hospital_pk          collection_week state  \
650      060014  2021-05-28T00:00:00.000    CO   
651      062012  2021-05-28T00:00:00.000    CO   
652      060011  2021-05-28T00:00:00.000    CO   
653      060028  2021-05-28T00:00:00.000    CO   
654      060064  2021-05-28T00:00:00.000    CO   
655      060032  2021-05-28T00:00:00.000    CO   
656      062009  2021-05-28T00:00:00.000    CO   
657      060107  2021-05-28T00:00:00.000    CO   

                                hospital_name    zip hospital_subtype  \
650     PRESBYTERIAN ST LUKE'S MEDICAL CENTER  80218       Short Term   
651          PAM SPECIALTY HOSPITAL OF DENVER  80204        Long Term   
652              DENVER HEALTH MEDICAL CENTER  80204       Short Term   
653                     SAINT JOSEPH HOSPITAL  80218       Short Term   
654  CENTURA HEALTH-PORTER ADVENTIST HOSPITAL  80210       Short Term   
655                       ROSE MEDICAL CENTER  80220       Short Term   
656                   KINDRED HOSPITAL

In [100]:
vaccinated =[]
for index, row in results_df.iterrows():
    total=float(row['total_personnel_covid_vaccinated_doses_none_7_day'])+float(row['total_personnel_covid_vaccinated_doses_one_7_day'])+float(row['total_personnel_covid_vaccinated_doses_all_7_day'])
    part=float(row['total_personnel_covid_vaccinated_doses_one_7_day'])+float(row['total_personnel_covid_vaccinated_doses_all_7_day'])
    vaccinated.append((part/total))
print(vaccinated)

[nan, 71.47435897435898, 92.79748136725777, 82.78114792347176, nan, nan, 93.71584699453553, 91.78972238629652]


In [154]:
InpatientBeds =[]
vaccinated =[]
Longitude = []
Latitude = []
addresses=[]
names=[]
for index, row in results_df.iterrows():
    total=float(row['inpatient_beds_7_day_avg'])
    part=float(row['inpatient_beds_used_7_day_avg'])
    if part>0 :
        InpatientBeds.append((part/total))
    else :
       InpatientBeds.append(0) 
    if isinstance(row['geocoded_hospital_address'],dict):
        Point=row['geocoded_hospital_address'].get("coordinates")
        splitPoint=str(Point).split(',')
        Longitude.append(splitPoint[0].strip("["))
        Latitude.append(splitPoint[1].strip("]"))
    else:
        Longitude.append('-104.9645')
        Latitude.append('39.7462')
    total=float(row['total_personnel_covid_vaccinated_doses_none_7_day'])+float(row['total_personnel_covid_vaccinated_doses_one_7_day'])+float(row['total_personnel_covid_vaccinated_doses_all_7_day'])
    part=float(row['total_personnel_covid_vaccinated_doses_one_7_day'])+float(row['total_personnel_covid_vaccinated_doses_all_7_day'])
    vaccinated.append((part/total))
    address=row['address']
    addresses.append(address)
    name=row['hospital_name']
    names.append(name)
        
print(Longitude, Latitude)
print(InpatientBeds)
print(vaccinated)
print(addresses)
print(names)

['-104.967175', '-105.035815', '-104.990066', '-104.972135', '-104.973482', '-104.934011', '-104.9645', '-104.942892'] [' 39.746089', ' 39.743568', ' 39.72823', ' 39.745722', ' 39.67064', ' 39.731076', '39.7462', ' 39.740019']
[0.3788912579957356, 0.7714285714285715, 0.9329058850469429, 0.5760445682451254, 0.2708333333333333, 0.610593220338983, 0.9449901768172888, 0]
[nan, 0.7147435897435898, 0.9279748136725777, 0.8278114792347177, nan, nan, 0.9371584699453552, 0.9178972238629651]
['1719 E 19TH AVE', '1690 N MEADE ST', '777 BANNOCK ST', '1375 E 19TH AVE', '2525 S DOWNING ST', '4567 E 9TH AVE', '1920 N HIGH ST', '1400 JACKSON ST']
["PRESBYTERIAN ST LUKE'S MEDICAL CENTER", 'PAM SPECIALTY HOSPITAL OF DENVER', 'DENVER HEALTH MEDICAL CENTER', 'SAINT JOSEPH HOSPITAL', 'CENTURA HEALTH-PORTER ADVENTIST HOSPITAL', 'ROSE MEDICAL CENTER', 'KINDRED HOSPITAL-DENVER', 'NATIONAL JEWISH HEALTH']


In [155]:
FinalHospitalSheet={
    'Name':names,
    'Address':addresses,
    'Longitude':Longitude,
    'Latitude':Latitude,
    'Occupancy':InpatientBeds,
    'VaccinatedPersonnnel':vaccinated
}
print(FinalHospitalSheet)

{'Name': ["PRESBYTERIAN ST LUKE'S MEDICAL CENTER", 'PAM SPECIALTY HOSPITAL OF DENVER', 'DENVER HEALTH MEDICAL CENTER', 'SAINT JOSEPH HOSPITAL', 'CENTURA HEALTH-PORTER ADVENTIST HOSPITAL', 'ROSE MEDICAL CENTER', 'KINDRED HOSPITAL-DENVER', 'NATIONAL JEWISH HEALTH'], 'Address': ['1719 E 19TH AVE', '1690 N MEADE ST', '777 BANNOCK ST', '1375 E 19TH AVE', '2525 S DOWNING ST', '4567 E 9TH AVE', '1920 N HIGH ST', '1400 JACKSON ST'], 'Longitude': ['-104.967175', '-105.035815', '-104.990066', '-104.972135', '-104.973482', '-104.934011', '-104.9645', '-104.942892'], 'Latitude': [' 39.746089', ' 39.743568', ' 39.72823', ' 39.745722', ' 39.67064', ' 39.731076', '39.7462', ' 39.740019'], 'Occupancy': [0.3788912579957356, 0.7714285714285715, 0.9329058850469429, 0.5760445682451254, 0.2708333333333333, 0.610593220338983, 0.9449901768172888, 0], 'VaccinatedPersonnnel': [nan, 0.7147435897435898, 0.9279748136725777, 0.8278114792347177, nan, nan, 0.9371584699453552, 0.9178972238629651]}


In [156]:
Finaldf=pd.DataFrame.from_dict(FinalHospitalSheet)
print(Finaldf)

                                       Name            Address    Longitude  \
0     PRESBYTERIAN ST LUKE'S MEDICAL CENTER    1719 E 19TH AVE  -104.967175   
1          PAM SPECIALTY HOSPITAL OF DENVER    1690 N MEADE ST  -105.035815   
2              DENVER HEALTH MEDICAL CENTER     777 BANNOCK ST  -104.990066   
3                     SAINT JOSEPH HOSPITAL    1375 E 19TH AVE  -104.972135   
4  CENTURA HEALTH-PORTER ADVENTIST HOSPITAL  2525 S DOWNING ST  -104.973482   
5                       ROSE MEDICAL CENTER     4567 E 9TH AVE  -104.934011   
6                   KINDRED HOSPITAL-DENVER     1920 N HIGH ST    -104.9645   
7                    NATIONAL JEWISH HEALTH    1400 JACKSON ST  -104.942892   

     Latitude  Occupancy  VaccinatedPersonnnel  
0   39.746089   0.378891                   NaN  
1   39.743568   0.771429              0.714744  
2    39.72823   0.932906              0.927975  
3   39.745722   0.576045              0.827811  
4    39.67064   0.270833                   

In [158]:
Finaldf.to_csv (r'C:\Users\pragyajha\OasisDataSets\Hospital.CSV', index = None, header=True) 

In [160]:
import requests

url = "https://besttime.app/api/v1/forecasts/live"

params = {
    'api_key_private': 'pri_63197f62b217434ab16286fce1031b9a',
    'venue_name': 'Safeway',
    'venue_address': '2660 Federal Blvd, Denver'
}

response = requests.request("POST", url, params=params)
print(response.json())

{'status': 'OK', 'analysis': {'venue_forecasted_busyness': 65, 'venue_forecasted_busyness_available': True, 'venue_live_busyness': 45, 'venue_live_busyness_available': True, 'venue_live_forecasted_delta': -20, 'hour_start': 19, 'hour_start_12': '7PM', 'hour_end': 20, 'hour_end_12': '8PM'}, 'venue_info': {'venue_current_gmttime': 'Sunday 2021-06-13 01:26AM', 'venue_current_localtime': 'Saturday 2021-06-12 07:26PM', 'venue_id': 'ven_5967334476674b39304c7a5263496234685a35366f596c4a496843', 'venue_name': 'Safeway', 'venue_address': '2660 N Federal Blvd Denver, CO 80211 United States', 'venue_timezone': 'America/Denver', 'venue_open': 'Open', 'venue_dwell_time_min': 20, 'venue_dwell_time_max': 20, 'venue_dwell_time_avg': 20}}


In [3]:
import pandas as pd
food_stores_df= pd.read_csv (r'food_stores.csv')
print (food_stores_df)

                      STORE_TYPE                   STORE_NAME  \
0                   Dollar Store                  Dollar Tree   
1                   Dollar Store                 Dollars City   
2            Small Grocery Store        Leever's / Save A Lot   
3              Convenience Store                     7-Eleven   
4              Convenience Store          For Every Day Store   
..                           ...                          ...   
583            Convenience Store                       Aztlan   
584  Discount Merchandise Market          Discount Empire Inc   
585          Small Grocery Store  Walmart Neighborhood Market   
586   Convenience Store With Gas                       Z-Mart   
587           Closed Supermarket                      Safeway   

           ADDRESS_LINE1    CITY STATE      ZIP   longitude   latitude  \
0      5190 Federal Blvd  Denver    CO  80221.0 -105.024782  39.790432   
1          4855 Pecos St  Denver    CO  80221.0 -105.007082  39.784801 

In [5]:
import requests
url = "https://besttime.app/api/v1/forecasts/live"
StoreOccupancy=[]
for index, row in food_stores_df.iterrows():
    types = row['STORE_TYPE']
    if (types=='Supermarket') or (types=='Small Grocery Store'):
        name=row['STORE_NAME']
        addy=row['ADDRESS_LINE1']
        params = {
        'api_key_private': 'pri_63197f62b217434ab16286fce1031b9a',
        'venue_name': name,
        'venue_address': addy
        }
        responses=requests.request("POST", url, params=params)
        if isinstance(responses,dict):
            BestTimeResponse_df = pd.DataFrame.from_dict(responses.json())
            if(BestTimeResponse_df['analysis']['venue_live_busyness_available']):
                StoreOccupancy.append(BestTimeResponse_df['analysis']['venue_live_busyness'])
            else:
                StoreOccupancy.append('nan') 
print(StoreOccupancy)

[]
